###  Load and Prepare Data



In [3]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Define parameters
vocab_size = 10000
embedding_dim = 100
maxlen = 100

# Load and prepare the dataset
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocab_size)
X_train_pad = pad_sequences(X_train, maxlen=maxlen)
X_test_pad = pad_sequences(X_test, maxlen=maxlen)


### Define the Model



In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Embedding

# Build the model
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim),
    Conv1D(filters=64, kernel_size=5, activation='relu'),
    MaxPooling1D(pool_size=4),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

### Train the Model



In [5]:
# Train the model
history = model.fit(X_train_pad, y_train, epochs=5, batch_size=32, validation_split=0.2)


Epoch 1/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 11s 15ms/step - accuracy: 0.6819 - loss: 0.5463 - val_accuracy: 0.8422 - val_loss: 0.3438
Epoch 2/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - accuracy: 0.9250 - loss: 0.1983 - val_accuracy: 0.8568 - val_loss: 0.3374
Epoch 3/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - accuracy: 0.9842 - loss: 0.0579 - val_accuracy: 0.8518 - val_loss: 0.4832
Epoch 4/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 9s 15ms/step - accuracy: 0.9985 - loss: 0.0084 - val_accuracy: 0.8516 - val_loss: 0.6414
Epoch 5/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - accuracy: 1.0000 - loss: 7.4416e-04 - val_accuracy: 0.8536 - val_loss: 0.7378


### Save the Model in Keras Format

In [6]:
# Save the model in Keras format
model.save('light_weighted_cnn_imdb_model.keras')


### Load the Model and Compile



In [7]:
# Load the model
loaded_model = tf.keras.models.load_model('light_weighted_cnn_imdb_model.keras')

# Compile the loaded model again
loaded_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


### Evaluate the Model

In [8]:
# Evaluate the model
loss, accuracy = loaded_model.evaluate(X_test_pad, y_test)
print(f'Test Accuracy: {accuracy:.4f}')


782/782 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.8459 - loss: 0.7710
Test Accuracy: 0.8443


### Make Predictions

In [9]:
# Predict new data
sample_reviews = ["This movie was fantastic!", "I did not enjoy this film."]
sample_reviews_seq = imdb.get_word_index()
sample_reviews_encoded = [[sample_reviews_seq.get(word, 0) for word in review.lower().split()] for review in sample_reviews]
sample_reviews_pad = pad_sequences(sample_reviews_encoded, maxlen=maxlen)

predictions = loaded_model.predict(sample_reviews_pad)
predicted_classes = ['Positive' if pred > 0.5 else 'Negative' for pred in predictions]
print(predicted_classes)


1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
['Negative', 'Negative']
